In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma


In [758]:
%run /Users/enzo-macmini/auction_alg_ITU/parallel_auction_ITU.ipynb

In [1034]:
n_small = 3
m_small = 3
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (4,24) , random_seed= 778, lbs =(0,0), tol = 1e-1)

In [1035]:
example_small.B_ij = np.ones([3,3])
example_small.A_ij = np.ones([3,3])
example_small.B_ij [:,-1] = 0


In [2302]:
def iteration(self,i_ent, mu, V_j, perm_unassigned = None):
    C  = np.zeros(self.n, dtype= bool)
    C_i_ent = np.zeros(self.n, dtype= bool)
    C[i_ent] = 1
    B = np.ones(self.m, dtype= bool)
    L_j = np.ones(self.m, dtype= int) *(self.n + 1)
    d_j = np.ones(self.m + 1, dtype= int) * np.inf

    i_j_diffs = np.zeros([self.n, self.m])

    iter = 0
    while np.any(C == 1):
        iter += 1
        i = np.argmax(C)
        print(f"iter {iter}, i: {i}")
        C[i] = 0
        C_i_ent[i] = 1
 
        U_ij_s = self.get_U_ij(V_j, i)[0]
        #print(U_ij_s)
     
        π_i = np.max(U_ij_s)
        #print(π_i)
        if π_i < self.lb_U:
            print("perm unassigned")
            perm_unassigned[i] = 1
            mu[i,:] = 0
            return mu, V_j, perm_unassigned
        
        
        suboptimal_js = U_ij_s < π_i 
      
        A_ij_star_max = np.max(self.A_ij[i,~suboptimal_js ])

        i_j_diffs[i, suboptimal_js] =  (π_i - U_ij_s[suboptimal_js] ) /A_ij_star_max


        d_j[:-1][suboptimal_js] = np.minimum(d_j[:-1][suboptimal_js], (π_i - U_ij_s[suboptimal_js] ) /A_ij_star_max) 
        d_j[-1] = np.minimum(  d_j[-1],   (π_i - self.lb_U) / A_ij_star_max)
      
        unissigned_optimal_js = np.where((mu.sum(0) == 0) * (π_i == U_ij_s))[0]
        
        if len(unissigned_optimal_js)> 0 :

            i_t = i
            j_t = unissigned_optimal_js[0]
      
            while i_t != i_ent :
 
                j_t_plus_1 = np.where(mu[i_t,: ]>0)[0][0]
          
                mu[i_t,: ] = 0
                mu[i_t, j_t] = 1
           
                w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, i_t)[0], [self.lb_U]]))[-2]
         
                V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)

                j_t = j_t_plus_1
                i_t = L_j[j_t]
           

            w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, i_t)[0], [self.lb_U]]))[-2]
            mu[i_t,: ] = 0
            mu[i_t, j_t] = 1

            V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)
            #print(f"j : {j}")
            #print(L_j)
            print("augmentation")
            return mu, V_j, perm_unassigned
            
        
        for j in np.where(B * (~suboptimal_js) >0)[0]:
            
            #U_ij = self.get_U_ij(V_j, i, j)[0]
            if mu[:,j].sum() == 1:
                i_j = np.argmax(mu[:,j])
                B[j] = 0
                C[i_j] = 1
                
                L_j[j] = i if L_j[j] == self.n + 1 else None
                #print(f"j : {j}")
                #print(L_j)
                print(f"i:{i} -> j:{j} ")


    matrix = i_j_diffs[ :, B * (d_j[:-1] < np.inf)]  
    masked_matrix = np.ma.masked_where(matrix <= 0, matrix)

    argmin_pos = np.where(masked_matrix == np.min(masked_matrix))
    #argmin_pos = np.unravel_index(np.argmin(masked_matrix), masked_matrix.shape)

    
    # construct j_low's
    j_low = np.zeros(self.m, dtype= bool)
    for i_low in np.unique(argmin_pos[0]):
    
        U_ij_s = self.get_U_ij(V_j, i_low)[0]
        π_i = np.max(U_ij_s)
        optimal_js = U_ij_s == π_i 
    
        j_low[optimal_js] = self.A_ij[i_low,optimal_js] == np.max(self.A_ij[i_low,optimal_js])
     


    return C_i_ent, B * (d_j[:-1] < np.inf), d_j, j_low

OneToOneITU.iteration = iteration

In [2046]:
def interim_auction(self,C, O, j_low, r, V_j ):

    pi_ij = np.zeros([C.sum(), self.m+1])

    V_j[j_low] += r

    unassigned = pi_ij.sum(1) == 0
    n_unassigned = unassigned.sum()
    iter = 0
    someone_unhappy = True

    while pi_ij.sum() < C.sum() or someone_unhappy:

        U_ij = np.concatenate((self.get_U_ij(V_j, C), np.ones((C.sum(),1)) * self.lb_U), axis = 1) 
        #print(U_ij.round(3))
        
        

        sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
        w_i = U_ij[np.arange(n_unassigned),sorted_w_id[:,0] ]
        π_i = np.max(U_ij, axis = 1)
    
        #pi_ij[:,-1] = π_i == self.lb_U 



        ij_optimal = U_ij == π_i[:, None]
        #print(ij_optimal)

        offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:] ) * ij_optimal[:,:-1]
     
        free_i = np.ones(C.sum(), dtype= bool) 
        for j in np.arange(self.m):
            if np.any(ij_optimal[:,j]>0) and len(offers[:,j][free_i])>0:
                best_offer = np.max(offers[:,j] [free_i] )
                i_j = np.argmax(offers[:,j] + (1- free_i)*(-1e+200)+ (1- ij_optimal[:,j])*(-1e+200))
        
                pi_ij[:, j] = 0 
                pi_ij[i_j, :] = 0 
                pi_ij[i_j, j] = 1 
                free_i[i_j] = 0
                V_j[j] += best_offer 

        out_offers = ij_optimal[:,-1]
        pi_ij[:,-1] = out_offers *  free_i

       
        pi_i = np.max(self.get_U_ij(V_j, C), axis= 1)
        id_i,  id_j = np.where(pi_ij[:,:-1] >0)
        #print(np.where(pi_ij[:,:-1] >0))
        happy_i = np.zeros(C.sum(), dtype= bool) 
        Values = self.get_U_ij(V_j,C )
        happy_i[id_i]  = Values[id_i, id_j] >= pi_i[id_i]

        unassigned = np.where(pi_ij[:,-1] >0)[0]
        happy_i[unassigned] = pi_i[unassigned] <= self.lb_U
    
        someone_unhappy = np.any(happy_i == False)

        iter += 1 
        if iter == 10:
            break

    return pi_ij, V_j, happy_i, iter

OneToOneITU.interim_auction =interim_auction
        

In [2359]:
mu_ex = np.zeros([n_small,m_small])
V_ex = np.ones(example_small.m) * example_small.lb_V
perm_unassigned = np.zeros(example_small.n, dtype= bool)

In [2360]:
example_small.B_ij

array([[89, 34, 37, 40, 44, 35, 22, 36, 41, 93],
       [ 8, 38, 59, 10, 11, 49, 31, 68, 34, 52],
       [34,  5, 49, 38, 17, 76, 71, 85, 70, 59],
       [47, 90, 50,  8, 99, 24, 56, 51, 83, 88],
       [72, 43,  7, 57, 98, 97, 81, 26, 30, 20],
       [ 8,  3, 81, 74, 39, 20, 72, 62, 13, 27],
       [ 2, 13, 57, 70, 69, 21, 84, 91, 44, 45],
       [86, 29, 60,  6, 95,  6, 43, 35,  9, 78],
       [88, 39, 98, 56, 51, 57,  6, 72, 84, 17],
       [85,  6,  0, 16, 12, 63, 84, 66,  4, 86]])

In [2361]:
# #C_i = np.zeros(example_small.n, dtype= bool)

# for p in range(1):

#     i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
#     iter_values = example_small.iteration(i_ent, mu_ex, V_ex )

#     if len(iter_values) == 2:
#         mu_ex, V_ex = iter_values

#     else:
#         C_i, B, d ,j_low= iter_values
#         print(B)
#         B_with_out = np.concatenate( (B, [True]) )
#         r = d[B_with_out]
#         #V_ex[~B] += np.min(d[B_with_out])

#         print(d)
#         print(B)
#         mu_ex[C_i,:] = 0
#         busy = np.zeros(example_small.m, dtype= bool)
#         for i in np.where(C_i > 0)[0]:
#             j_i = np.argmax( example_small.get_U_ij(V_ex, i) )
#             #print(j_i)
#             if busy[j_i] == False:
#                 mu_ex[:,j_i] = 0
#                 mu_ex[i,j_i] = 1
#                 busy[j_i] = 1


       

#         #print(B)
#         print(f"d:{np.min(d[B_with_out])}")
#         print(f"iteration total:{p}")
    
        

#     perm_unassigned = np.all(example_small.get_U_ij(V_ex , np.arange(example_small.n)) <= example_small.lb_U , axis = 1)
# mu_ex[perm_unassigned,:] = 0

In [2371]:
#C_i = np.zeros(example_small.n, dtype= bool)

for p in range(1):

    i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
    iter_values = example_small.iteration(i_ent, mu_ex, V_ex ,perm_unassigned)

    if len(iter_values) == 3:
        mu_ex, V_ex , perm_unassigned = iter_values

        print(mu_ex)
    else:
        print("AAAA")
        C_i, B, d ,j_low= iter_values
        
        B_with_out = np.concatenate( (B, [True]) )
        r = d[B_with_out]
        #V_ex[~B] += np.min(d[B_with_out])
        pi_ij, V_ex, happy_i, iter = example_small.interim_auction(C_i, ~B, j_low,  np.min(r),V_ex )

        # this should be revoming exactly one 0
        print(pi_ij)
        #pi_ij[:,:-1][:,mu_ex[~C_i,:].sum(0) == 1 ]= 0


        mu_ex[C_i,:] = pi_ij[:,:-1]
        perm_unassigned[C_i] = pi_ij[:,-1]


    
   
# mu_ex[perm_unassigned,:] = 0

iter 1, i: 9
i:9 -> j:6 
iter 2, i: 6
i:6 -> j:7 
iter 3, i: 1
AAAA
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [2372]:
V_ex

array([ 35.76      ,   0.        ,  40.        ,   4.44952563,
        41.4       ,  12.44444444, 151.06227106,  75.97802198,
         0.        ,   2.41758242])

In [2373]:
mu_ex

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [2375]:
(example_small.get_U_ij(V_ex, np.ones(example_small.n, dtype= bool))[6,:]).round(2)

array([-79.06,  13.  , -74.03,  59.62,  29.57,   8.03,  56.97,  59.62,
        44.  ,  39.02])

In [2310]:
mu_ex.sum(0)

array([1., 0., 1., 1., 1., 1., 0., 1., 1., 1.])

In [2123]:
def coop_ITU_auction(self):
    mu_ij = np.zeros([self.n,self.m])
    V_j = np.ones(self.m) * self.lb_V
    perm_unassigned = np.zeros(self.n, dtype= bool)
    iters = 0 
    while np.sum((mu_ij.sum(1) == 0) * (~perm_unassigned)) > 0:
        iters += 1
        i_ent = np.where((mu_ij.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
        iter_values = self.iteration(i_ent, mu_ij, V_j ,perm_unassigned)
        #print(iter_values)
        if len(iter_values) == 3:
            mu_ij, V_j ,perm_unassigned = iter_values

        else:
            print("AAAA")
            C_i, B, d ,j_low= iter_values
            
            B_with_out = np.concatenate( (B, [True]) )
            r = d[B_with_out]
            #V_j[~B] += np.min(d[B_with_out])
            pi_ij, V_j, happy_i, iter = self.interim_auction(C_i, ~B, j_low,  np.min(r),V_j )

            # this should be revoming exactly one 0
            pi_ij[:,:-1][:,mu_ij[~C_i,:].sum(0) == 1 ]= 0


            mu_ij[C_i,:] = pi_ij[:,:-1]
            perm_unassigned[C_i] = pi_ij[:,-1]


        

    #mu_ij[perm_unassigned,:] = 0



    id_i , id_j  =  np.where(mu_ij == 1)
    
    U_i = np.ones(example_small.n) * example_small.lb_U
    U_i[id_i] = example_small.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j] 
    print(iters)
    return mu_ij, U_i , V_j




OneToOneITU.coop_ITU_auction = coop_ITU_auction

In [2124]:
# def ITU_auction(self):
#     mu_ij = np.zeros([self.n,self.m])
#     V_j = np.ones(self.m) * self.lb_V
#     perm_unassigned = np.zeros(self.n, dtype= bool)
#     iters = 0
#     # for p in range(10000):

#     while np.sum((mu_ij.sum(1) == 0) * (~perm_unassigned)) > 0:
#         iters += 1
#         i_ent = np.where((mu_ij.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
#         iter_values = self.iteration(i_ent, mu_ij, V_j )

#         if len(iter_values) == 2:
#             mu_ij, V_j = iter_values

#         else:
#             print("AAAA")
#             C_i, B, d ,j_low= iter_values
            
#             B_with_out = np.concatenate( (B, [True]) )
#             r = d[B_with_out]
#             #V_j[~B] += np.min(d[B_with_out])
#             pi_ij, V_j, happy_i, iter = self.interim_auction(C_i, ~B, j_low,  np.min(r),V_j )

#             # this should be revoming exactly one 0
#             pi_ij[:,:-1][:,mu_ij[~C_i,:].sum(0) == 1 ]= 0


#             mu_ij[C_i,:] = pi_ij[:,:-1]
#             perm_unassigned[C_i] = pi_ij[:,-1]


    
        

#     perm_unassigned = np.all(self.get_U_ij(V_j , np.arange(self.n)) <= self.lb_U , axis = 1)
#     mu_ij[perm_unassigned,:] = 0

#     id_i , id_j  =  np.where(mu_ij == 1)
    
#     U_i = np.ones(example_small.n) * example_small.lb_U
#     U_i[id_i] = example_small.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j] 
#     return mu_ij, U_i , V_j



#     # id_i , id_j  =  np.where(mu_ij == 1)
    
#     # U_i = np.ones(self.n) * self.lb_U
#     # U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]   

#     # return mu_ij,U_i, V_j, perm_unassigned * 1, iters

# OneToOneITU.ITU_auction = ITU_auction

In [2161]:
n_small = 10
m_small = 10
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (100,100) , random_seed= 588845, lbs =(2,0), tol = 0)

In [2152]:
# mu_ex , U_ex,  V_ex , perm_ , iters= example_small.ITU_auction()

In [2162]:
mu_ex , U_ex,  V_ex  = example_small.coop_ITU_auction()

iter 1, i: 0
augmentation
iter 1, i: 1
augmentation
iter 1, i: 2
augmentation
iter 1, i: 3
augmentation
iter 1, i: 4
iter 2, i: 2
augmentation
iter 1, i: 5
augmentation
iter 1, i: 6
iter 2, i: 2
AAAA
iter 1, i: 7
augmentation
iter 1, i: 8
iter 2, i: 5
augmentation
iter 1, i: 9
iter 2, i: 6
iter 3, i: 1
AAAA
iter 1, i: 6
iter 2, i: 1
iter 3, i: 0
iter 4, i: 5
AAAA
iter 1, i: 5
iter 2, i: 0
augmentation
iter 1, i: 9
iter 2, i: 6
iter 3, i: 2
AAAA
iter 1, i: 9
iter 2, i: 6
iter 3, i: 2
augmentation
14


In [2154]:
mu_ex.sum()

10.0

In [2292]:
id_i , id_j  =  np.where(mu_ex == 1)
    
U_ex = np.ones(example_small.n) * example_small.lb_U
U_ex[id_i] = example_small.get_U_ij(V_ex, id_i)[np.arange(len(id_i)), id_j]  

In [2293]:
example_small.check_all((mu_ex , U_ex , V_ex ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 10 over 10

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [2164]:
example_small.get_U_ij(V_ex , np.arange(5)) . round(3)

array([[  -3.07 ,   34.   ,   20.478,   23.921,   -0.934,    5.667,
        -520.249,  -71.667,    9.485,   41.   ],
       [ -22.257,   38.   ,   25.769,    3.269,   -5.327,   18.368,
         -32.794,   38.   , -471.56 ,   27.597],
       [ -59.039,    5.   , -118.619,    3.699,  -48.981,   -1.156,
         -47.381, -546.125,   46.287,   46.287],
       [   6.018,   90.   ,   14.898,    1.92 ,   90.   ,   16.944,
           0.661,  -36.237,   64.168,   76.008],
       [  41.661,   43.   ,  -55.609,   27.41 ,   29.   ,   81.   ,
          60.084,  -52.818,   25.798,  -83.452]])

In [2503]:
example_small. check_dual_feas(U_ex,V_ex, output = True)

<IPython.core.display.Math object>

(array([[-13.8351983 ,   0.        , -10.92857143,   0.        ,
          -3.28571429,   0.        ,   0.        ,   0.        ,
         -13.93650794,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.   

In [2038]:
mu_ex[0,:].sum()

1.0